# Make  spectra

- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab/IN2P3/CNES, DESC-LSST fullmember, behalf AuxTel Teal VERA C. Rubin observatory
- creation date : March 29th 2021
- version 

# Imports

In [53]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
import pandas as pd

import matplotlib.ticker                         # here's where the formatter is
import os
import re

plt.rcParams["figure.figsize"] = (16,4)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

In [54]:
from astropy.io import fits
from spectractor import parameters
from spectractor.extractor.extractor import Spectractor

In [55]:
dir_spectra="/Users/dagoret/DATA/AuxTelData2021/holo/FlipCleans"
filename_image="Cor_holo4_003_RG610_HD75519_2021-03-11_296.fits"

In [56]:
filename=os.path.join(dir_spectra,filename_image)
output_directory="./outputs/"
guess = [1762,2209]
disperser_label = "holo4_003"
config = "config/auxtel_quicklook.ini"
#config = "config/auxtel.ini"
target = "HD75519"

In [57]:
hdu = fits.open(filename)

In [58]:
hdu.info()

Filename: /Users/dagoret/DATA/AuxTelData2021/holo/FlipCleans/Cor_holo4_003_RG610_HD75519_2021-03-11_296.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  Segment10     1 PrimaryHDU     121   ()      
  1                1 ImageHDU         7   (4072, 4000)   float64   


In [59]:
hdu0=hdu[0]

In [60]:
hdu0.header

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                    8 / array data type                                
NAXIS   =                    0 / number of array dimensions                     
EXTEND  =                    T                                                  
ORIGIN  = 'LSST DM Header Service'                                              
TIMESYS = 'TAI     '                                                            
DATE    = '2021-03-12T02:53:12.435'                                             
DATE-OBS= '2021-03-12T02:53:21.563'                                             
DATE-BEG= '2021-03-12T02:53:21.563'                                             
DATE-END= '2021-03-12T02:53:46.800'                                             
MJD     =      59285.120282809                                                  
MJD-OBS =     59285.1203884634                                                  
MJD-BEG =     59285.12038846

In [61]:
spectrum = Spectractor(filename, output_directory, guess=guess, target_label=target, disperser_label=disperser_label, config=config)

KeyError: "Keyword 'CD2_1' not found."